
# **Practice 2: Trajectory-Based Metaheuristics – Tabu Search**

This practice focuses on implementing **Tabu Search**, a trajectory-based metaheuristic, to solve optimization problems efficiently. You will start by implementing a **basic version** of the algorithm and then have the option to develop an **improved version** with enhanced performance.

## **Objectives**
- Understand the principles of **Tabu Search**.
- Implement a **basic** Tabu Search algorithm.
- Analyze the impact of different **neighborhood structures** and **tabu list strategies**.
- Compare the effectiveness of the **basic** and **improved** implementations.



###    _Nombre Apellidos_



## **Instructions**
Follow the provided steps carefully, executing the code in sequence and analyzing the results. Throughout the practice, you will find **questions** that must be answered in the designated section at the bottom of the document.

Just like in **Practice 1**, we will use a **Jupyter Notebook** for this practice.

As you already know, it allows us to execute code cells step by step, as well as automatically generate a well-formatted report of the practice. Nevertheless, here are some brief instructions on how it works:

* You can add a cell using the **"Insert"** button in the toolbar and change its type with **"Cell > Cell Type"**.
* To execute a code cell, select it and press the **"▶ Run"** button in the toolbar.
* To export the document to HTML, select **"File > Download as > HTML (.html)"**.

Follow this guide until the end. Execute the provided code step by step, understanding what you are doing and reflecting on the results. There will be questions interspersed throughout the guide—answer all of them in the designated section: **"Responses to the questionnaires"**. Please do not modify any line of code unless explicitly instructed to do so.

Do not forget to insert your **name and surname** in the top cell.

### **Implementation Requirements**
You will be required to implement **two versions** of the **Tabu Search** algorithm:
1. Implement a **mandatory** basic version of Tabu Search.
2. Optionally, propose and implement an **improved** version with modifications such as:
   - Different neighborhood selection strategies.
   - Variable tabu list size.
   - Diversification mechanisms to escape local optima.


Write the code for your solution(s) in the designated cells. Additionally, throughout the practice, several questions will be posed that you must answer in the lower section of the document. Feel free to include any necessary cells (for example, if you reference specific parts of your code) to properly answer the questions.


## Practice Submission

The submission deadline will be the one indicated in the Virtual Campus. The submission must consist of a single compressed file named `LASTNAME_FIRSTNAME_TabuSearch.zip`, containing the following files:

 * `LASTNAME_FIRSTNAME_TabuSearch.html`: An HTML file generated from exporting this Notebook, with the answered questions at the end of the document.
 * `LASTNAME_FIRSTNAME_TabuSearch.ipynb`: The source Jupyter Notebook file.
 * The data file(s) used for problem-solving.


 ---


## Python preliminaries


Here you have some Python functions that may be helpful in the near future while developing this practice.

For example, you can generate random numbers using the package `random`.

In [ ]:
import random

# we can create a random number between 1 and 10
random_number = random.randint(1, 10)
print(random_number)

# and random numbers between 0 and 1 following a uniform distribution
random_U = random.uniform(0, 1)
print(random_U)

You can generate vectors of fixed and random numbers that are also shuffled randomly, as illustrated below.

In [ ]:
vector = [x for x in range(1, 10)]
print("fixed vector", vector)

random.shuffle(vector)
print(vector)

random_vector = [random.randint(1, 10) for i in range(1, 10)]
print("random vector", random_vector)

random.shuffle(random_vector)
print(random_vector)


**NEW:** You can also generate a list of random numbers without repetition within a given range (a permutation of the range).

In [ ]:
print(random.sample(range(1,10), 9))

Another important set of functions comes from the math module. You can find a list of available functions at https://docs.python.org/3/library/math.html. Below are some usage examples.

In [ ]:
import math 

# number e raised to the specified power
e = math.exp(1)
print(e)

power2_e = math.exp(2)
print(power2_e)

# example of exponentiation
print(math.pow(e, 1))
print(math.pow(e, 2))

# example of the natural logarithm with base e
base = e
print(math.log(e))
print(math.log(e, base))


Finally, functions from the `time` module allow you to approximately measure the execution time of specific sections of code.

In [ ]:
import time
start_time = time.time()

sum = 0
for i in range(1000000):
    sum = sum * 1

print("---- %s segundos ----" % (time.time() - start_time))

# The Traveling Salesperson Problem (TSP) with Tabu Search

Once again, we will attempt to solve the **Traveling Salesperson Problem (TSP)**, but this time using the **Tabu Search algorithm**.

The objective of this practice is to **model and implement an intelligent agent** capable of solving the TSP using the **Tabu Search (TS) metaheuristic**. To achieve this, you will first implement the **basic algorithm** covered in the lecture and then evaluate whether introducing **modifications** to its design can improve the quality of the obtained solutions.


## Definition of the Traveling Salesperson Problem (TSP)

The **Traveling Salesperson Problem (TSP)** involves a salesperson who wants to sell a product and, to do so, needs to find the **shortest possible route** through the cities of their customers, visiting each city only once and **starting and ending** the journey in their own city (a circular route from the initial city).

Typically, the problem is represented using a **weighted graph** \( G = (N, A) \), where:
- \( N \) is the set of **\( n = |N| \) nodes** (cities).
- \( A \) is the set of **edges** connecting the nodes.
- Each edge \( (i, j) in A \) has an associated weight \( d_ij \), which represents the **distance** between cities \( i \) and \( j \).

The **TSP reduces to the problem of finding the minimum-length Hamiltonian circuit** in the graph \( G \). A solution to a TSP instance can be represented as a **permutation of city indices**, where the **order of visits** determines the **total travel cost** in terms of the total distance covered.

Since there are **\( n! \) possible permutations**, the problem belongs to the **NP category**, making it computationally expensive to solve as \( n \) increases. This means that solving instances with a large number of cities (**large \( n \)**) is impractical using **uninformed search strategies**. Instead, the problem can **benefit from metaheuristics**, allowing larger instances to be tackled while still obtaining **reasonably good solutions** within a feasible time.


### Preliminary Concepts

To facilitate your implementation work, we provide the **`Localizaciones`** class, which allows you to **load GPS locations** representing the vertices of the graph \( G \) of \( N \) cities. It also enables you to **transparently calculate** the distance between any pair of cities using the **[haversine formula](https://en.wikipedia.org/wiki/Haversine_formula)**, which accounts for the Earth's curvature when computing distances.

It is important to note that in the **haversine formula**, the coordinates must be expressed in **radians**.

First, import the **Python module** that accompanies this practice, which includes some **support functions** as well as the **data loading class**.


In [ ]:
from helpers_mod_sa import *

Inspect the location loading code using `psource(Localizaciones)`.

In [ ]:
psource (Localizaciones)

By default, the file `./data/grafo8cidades.txt` is loaded, which contains the **GPS coordinates** of **8 Galician cities**, with **Santiago de Compostela** being the first one. The first line of these files indicates the **number of cities** \( n \), while each of the following lines specifies the **coordinates** of each city, given as **GPS coordinates (latitude and longitude in degrees).**

You can load a different file by using the **`filename`** parameter, as shown below. If everything works correctly, the **first distance between city 0 and city 1 should be approximately 55 km**.

> ❗ **Important:** For this practice, **you must use** the file `./data/grafo100cidades.txt`, which contains the coordinates of **100 Galician municipalities**.

In [ ]:
g1=Localizaciones(filename='./data/grafo8cidades.txt')
print (g1.distancia(0,1))
g2=Localizaciones(filename='./data/grafos10_10/grafo_1.txt')
print (g2.distancia(0,1))
g3=Localizaciones(filename='./data/grafo100cidades.txt')
print(g3.distancia(0,1))


---

# P2.1: Basic Implementation of Tabu Search

In this section, you must develop a **basic version** of the **Tabu Search** algorithm to solve the **Traveling Salesperson Problem (TSP)** applied to the municipalities of Galicia. The specification of the algorithm will be highly detailed since the main objective of this first part is to ensure you have a **fully functional and verified implementation** that correctly solves the problem. As in **P1**, we assume that the **tour is circular** (starting and ending at the same municipality) and must include **N = 100** municipalities in Galicia.

To implement the algorithm, **review the algorithmic description** of the metaheuristic covered in the lecture (**See T1, slide 49 and related slides**).

### Design Considerations for the Basic Implementation
- **Solution Representation:** The solution is represented as an **ordered sequence (permutation)** **starting and ending at city 0**. That is, we use an ordered representation consisting of a sequence of numerical values representing each municipality **{0, 1, ..., 99}**. The starting and ending point is always **municipality 0**, so a valid solution **S** is represented as a permutation of the remaining values **{1, ..., 99}**.

- **Initial Solution:** A **completely random** permutation of the cities is generated as described in previous sections.

- **Neighborhood Generation Operator:** The **swap operator** is used to generate new neighboring solutions. The maximum number of different neighbors that can be generated from a given solution (all possible swaps) is:

  $$
  \sum_{i=1}^{L-1}i = \frac{L(L-1)}{2}
  $$

  Where **L** is the length of the solution.

- **Cost Function:** The total travel cost is computed as the **sum of the distances** in the tour while ensuring that the solution **starts and ends at city 0**. The distance is calculated based on the following three elements:
    - **Distance from city 0 to the first city in the solution:** `0 -> S[0]`
    - **Total distance traveled in the solution:** `S[0] -> S[1] -> ... -> S[-1]`
    - **Distance from the final city back to city 0:** `S[-1] -> 0`

- **Tabu List:** The **Tabu List (TL)** consists of **swap moves** `{i, j}` that generate the solutions forming the search trajectory. You must set **N = 100** as the **tabu tenure** parameter, meaning that the **tabu list will store N elements**. A move `{i, j}` will exit the tabu list after **N = 100** operations, making it permissible again.

- **Reinitialization Strategy:** If **100 consecutive iterations** pass without improving the best solution **\( S_{opt} \)** found so far, a **restart** is triggered from **\( S_{opt} \)**. This serves as an **intensification strategy**.  
  - The **tabu list is NOT reset** during reinitialization, allowing the exploration of previously unvisited neighbors due to prohibited swaps.
  - When restarting, the algorithm returns to **\( S_{opt} \)** and clears the tabu list.

- **Stopping Criterion:** The algorithm terminates after **10,000 iterations**.

### **Verification of Your Implementation**
To verify your implementation, **you must use** the **100-city Galician dataset** (`grafo100cidades.txt`).  
As an initial test to ensure the implementation works correctly, you can use the **8-city Galician dataset** (`grafo8cidades.txt`). The **optimal solution** found using an **informed search** such as **A*** is approximately **382 km**.


In [ ]:
# Write your code and responses to the questions in the cells at the end of this notebook

❓ **Question 1**: Briefly explain the relevant details of your code to help understand your implementation (e.g., code structure, functions, etc.).

❓ **Question 2**: The experimental part of this practice consists of performing **10 different runs** of your implementation and reporting the **mean and standard deviation** of:
   - The best solution obtained.
   - The iteration number at which it was reached.
   - The execution time of the algorithm.

🔹 **Note**: Be cautious when verifying your implementation, especially when using **large datasets**, such as the **USA cities problem**. If you run your algorithm for a **large number of iterations**, it may be useful to **measure execution time** to make informed decisions about where to set the iteration limit.


---

# P2.2: Improvements to the Tabu Search Algorithm (Optional)

In this section, the objective is to apply the algorithm you have just implemented to a **new dataset of 120 locations** extracted from the [50,000 historical places in the U.S. National Register](http://www.math.uwaterloo.ca/tsp/us/data.html), as described on the [Traveling Salesman Problem (TSP)](http://www.math.uwaterloo.ca/tsp/) website of the [Department of Combinatorics and Optimization](https://uwaterloo.ca/combinatorics-and-optimization/) at **University of Waterloo, CA** [(Prof. William Cook)](http://www.math.uwaterloo.ca/~bico/).

To **avoid excessive computation time**, we will limit the problem to **120 locations**, which are provided in the file **US120.txt**.

> 🔹 **Note**: If anyone wants to test the algorithm with the full dataset, the original text file can be downloaded from the [following link](http://www.math.uwaterloo.ca/tsp/us/files/us50000_latlong.txt).


### Goal: Improving the Tabu Search Algorithm
In this section, the objective is to **enhance** the previously developed algorithm based on the concepts covered in the lectures. You may modify any parameter or operator, such as:

- **Initial solution generation** (e.g., using a greedy initialization).
- **Tabu list management**, including an **aspiration criterion** (e.g., allowing a solution to be removed from the tabu list if it improves the best solution found so far).
- **Neighborhood generation operator**, such as:
  - Not considering all pairs of indices.
  - Changing the neighborhood generation operator.
- **Alternative intensification restart strategies**, such as:
  - Restarting from a random solution selected from the **N best solutions found so far**.
  - Restoring the tabu list.
- **Diversification strategy using long-term memory**, such as:
  - Implementing a **frequency matrix** (`frec`) that records the number of times each pair of cities appears consecutively in accepted solutions.
  - Using this frequency matrix to perform a greedy initialization on a **modified distance matrix**, which **penalizes frequently visited pairs** by increasing their distance artificially:

  $$
  D(i,j)_{MOD} = D(i,j) + \mu (D_{MAX} - D_{MIN}) \frac{frec(i,j)}{frec_{MAX}}
  $$

- **Strategic oscillation**: Alternating between **intensification** and **diversification** strategies.

This section is **optional** but provides an opportunity to explore **advanced techniques** that improve the efficiency and solution quality of Tabu Search for larger instances of the TSP.

In [ ]:
# Write your code and responses to the questions in the cells at the end of this notebook

❓ **Question 3**: Which improvements led to better results?  

Briefly explain the **enhancements or modifications** you implemented, how you implemented them, and why you believe they are beneficial for the problem. Present your **conclusions** along with the **results obtained** to support your analysis.

---

# Responses to the Questions

> **Reminder**: Do not forget to write your **name and surname** in the second cell of this document.  The responses to the questions must be **accompanied by the necessary implementations** to support your answers.

### **P1.1 Mandatory Specification**

This first part is **graded out of 6 points**. To complete it, you must:  
- **Implement the algorithm**, and  
- **Answer Questions 1 and 2** accordingly.  

The **evaluation** of this section will be based on the **combination of the implementation and the responses** to both questions.

❓ **Question 1**: Briefly explain the relevant details of your code to help understand your implementation (e.g., code structure, functions, etc.).  

💡 *Include all the cells you find necessary to make your explanation clear and easy to follow.*


In [ ]:
# Write your code here for the function that implements the Tabu Search
# Create as many cells as you find necessary to write your code
# Always document your code with comments like this

❓ **Question 2**: The experimental part of this practice consists of performing **10 different runs** of your implementation and reporting the **mean and standard deviation** of:
   - The best solution obtained.
   - The iteration number at which it was reached.
   - The execution time of the algorithm.

💡 *Include all the cells you find necessary to make your explanation clear and easy to follow.*



In [ ]:
# Write your code here for the function that implements the experimental part

### **Optional Specification (Improvements)**

This second part is **optional** and will be graded out of **4 points**. To complete it, you must:  
- **Implement improvements**,  
- **Show the code** where these improvements were implemented, and  
- **Explain the rationale** behind these improvements and the results obtained.

❓ **Question 3**: Which improvements led to better results?  

Briefly explain the **enhancements or modifications** you implemented, how you implemented them, and why you believe they are beneficial for the problem. Present your **conclusions** along with the **results obtained** to support your analysis.  

💡 *Include all the cells you find necessary to make your explanation clear and easy to follow.*

In [ ]:
# Write your code here for the function that implements the Tabu Search
# Create as many cells as you find necessary to write your code
# Always document your code with comments like this